# Introduction  

Let's investigate how we can use a Langchain SQL Agent to explore a database.

# Packages installation and import

In [1]:
!pip install -q -U langchain
!pip install -q -U openai

In [2]:
from langchain.agents import create_sql_agent 
from langchain.agents.agent_toolkits import SQLDatabaseToolkit 
from langchain.sql_database import SQLDatabase 
from langchain.llms.openai import OpenAI 
from langchain.agents import AgentExecutor 
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

# Setup the database connection

In [3]:
sqlite_path = "/kaggle/input/24169-pitchfork-reviews/data.sqlite3"
sqlite_uri = f"sqlite:///{sqlite_path}"

In [4]:
db = SQLDatabase.from_uri(sqlite_uri)

# Configure ChatOpenAI

We will need the connection token for OpenAI. We extract it from the secrets.

In [5]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

We initialize the LLM that will be used to power the SQLAgent.
For this experiment, we will use ChatGPT (`gpt-3.5-turbo`).

In [6]:
llm = ChatOpenAI(temperature=0, 
                 openai_api_key=user_secrets.get_secret("OPENAI_API_KEY"), 
                 model_name='gpt-3.5-turbo')

We initialize now the SQL toolkit with the LLM and with the database connection.

In [7]:
toolkit = SQLDatabaseToolkit(db=db, 
                             llm=llm)

# Prepare the SQL Agent

The Agent executor is initialized with `create_sql_agent` from Langchain, setting the LLM parameter, the toolkit parameter, and the agent type.
We want to be able to monitor the entire process, therefore we set the parameter `verbose` to `True`.

In [8]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

# Test the agent

Now we will test the agent with few simple tasks.

In [9]:
question = "Just list the tables in the database"
agent_executor.run(question)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: artist_review_map, artists, author_review_map, genre_review_map, label_review_map, reviews, tombstone_release_year_map, tombstones
Thought:I now know the tables in the database.
Final Answer: The tables in the database are artist_review_map, artists, author_review_map, genre_review_map, label_review_map, reviews, tombstone_release_year_map, and tombstones.

> Finished chain.


'The tables in the database are artist_review_map, artists, author_review_map, genre_review_map, label_review_map, reviews, tombstone_release_year_map, and tombstones.'

In [10]:
question = "What is the structure of table 'artists'"
agent_executor.run(question)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: artist_review_map, artists, author_review_map, genre_review_map, label_review_map, reviews, tombstone_release_year_map, tombstones
Thought:The 'artists' table exists in the database. I can now query the schema of the 'artists' table.
Action: sql_db_schema
Action Input: "artists"
Observation: 
CREATE TABLE artists (
	artist_id VARCHAR NOT NULL, 
	name VARCHAR NOT NULL, 
	artist_url VARCHAR, 
	PRIMARY KEY (artist_id)
)

/*
3 rows from artists table:
artist_id	name	artist_url
4847-avarus	Avarus	/artists/4847-avarus/
33420-pinegrove	Pinegrove	/artists/33420-pinegrove/
2438-le-tigre	Le Tigre	/artists/2438-le-tigre/
*/
Thought:The 'artists' table has the following structure:
- artist_id (VARCHAR)
- name (VARCHAR)
- artist_url (VARCHAR)

The table contains 3 rows with the columns artist_id, name, and artist_url.

Final Answer: The structure of the 'artists' table is:
- artist_id (VARCHAR)
- name (

"The structure of the 'artists' table is:\n- artist_id (VARCHAR)\n- name (VARCHAR)\n- artist_url (VARCHAR)"

Now we will start exploring the database using more advanced queries.

In [11]:
question = "How many artists are in the database"
agent_executor.run(question)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: artist_review_map, artists, author_review_map, genre_review_map, label_review_map, reviews, tombstone_release_year_map, tombstones
Thought:I can query the "artists" table to get the number of artists in the database.
Action: sql_db_schema
Action Input: "artists"
Observation: 
CREATE TABLE artists (
	artist_id VARCHAR NOT NULL, 
	name VARCHAR NOT NULL, 
	artist_url VARCHAR, 
	PRIMARY KEY (artist_id)
)

/*
3 rows from artists table:
artist_id	name	artist_url
4847-avarus	Avarus	/artists/4847-avarus/
33420-pinegrove	Pinegrove	/artists/33420-pinegrove/
2438-le-tigre	Le Tigre	/artists/2438-le-tigre/
*/
Thought:I can use the "artists" table to count the number of artists in the database.
Action: sql_db_query
Action Input: "SELECT COUNT(*) FROM artists"
Observation: [(11013,)]
Thought:I now know the final answer.
Final Answer: There are 11,013 artists in the database.

> Finished chain.


'There are 11,013 artists in the database.'

In [12]:
question = "Which artist has most reviews?"
agent_executor.run(question)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: artist_review_map, artists, author_review_map, genre_review_map, label_review_map, reviews, tombstone_release_year_map, tombstones
Thought:I can query the tables "artist_review_map" and "artists" to find the artist with the most reviews. I should check the schema of these tables to see which columns are relevant.
Action: sql_db_schema
Action Input: "artist_review_map, artists"
Observation: 
CREATE TABLE artist_review_map (
	review_url VARCHAR NOT NULL, 
	artist_id VARCHAR NOT NULL, 
	PRIMARY KEY (review_url, artist_id), 
	FOREIGN KEY(review_url) REFERENCES reviews (review_url), 
	FOREIGN KEY(artist_id) REFERENCES artists (artist_id)
)

/*
3 rows from artist_review_map table:
review_url	artist_id
/reviews/albums/9232-vesikansi/	4847-avarus
/reviews/albums/22823-elsewhere/	33420-pinegrove
/reviews/albums/4703-le-tigre/	2438-le-tigre
*/


CREATE TABLE artists (
	artist_id VARCHAR NOT NULL, 
	n

'Various Artists'

In [13]:
question = "How many reviews are in total?"
agent_executor.run(question)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: artist_review_map, artists, author_review_map, genre_review_map, label_review_map, reviews, tombstone_release_year_map, tombstones
Thought:I can query the "reviews" table to get the total number of reviews.
Action: sql_db_schema
Action Input: "reviews"
Observation: 
CREATE TABLE reviews (
	review_url VARCHAR NOT NULL, 
	is_standard_review BOOLEAN NOT NULL, 
	pub_date DATETIME NOT NULL, 
	body TEXT NOT NULL, 
	PRIMARY KEY (review_url)
)

/*
3 rows from reviews table:
review_url	is_standard_review	pub_date	body
/reviews/albums/9232-vesikansi/	True	2006-07-27 06:00:04	Over the past few years, several of Finland's various interrelated underground acts-- Kemialliset Ys
/reviews/albums/22823-elsewhere/	True	2017-01-27 06:00:00	It's hard not to root for Pinegrove. Even before they raised more than $21,000 for Planned Parenthoo
/reviews/albums/4703-le-tigre/	True	1999-10-26 05:00:03	Kathleen Hanna 

'There are 24,169 reviews in total.'

In [14]:
question = "How many reviews have the artist with most of the reviews?"
agent_executor.run(question)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: artist_review_map, artists, author_review_map, genre_review_map, label_review_map, reviews, tombstone_release_year_map, tombstones
Thought:I can query the tables "artist_review_map" and "artists" to find the artist with the most reviews. Then I can query the "reviews" table to count the number of reviews for that artist.
Action: sql_db_schema
Action Input: "artist_review_map, artists, reviews"
Observation: 
CREATE TABLE artist_review_map (
	review_url VARCHAR NOT NULL, 
	artist_id VARCHAR NOT NULL, 
	PRIMARY KEY (review_url, artist_id), 
	FOREIGN KEY(review_url) REFERENCES reviews (review_url), 
	FOREIGN KEY(artist_id) REFERENCES artists (artist_id)
)

/*
3 rows from artist_review_map table:
review_url	artist_id
/reviews/albums/9232-vesikansi/	4847-avarus
/reviews/albums/22823-elsewhere/	33420-pinegrove
/reviews/albums/4703-le-tigre/	2438-le-tigre
*/


CREATE TABLE artists (
	artist_id VARC

'782'

In [15]:
question = "Which are the genres?"
agent_executor.run(question)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: artist_review_map, artists, author_review_map, genre_review_map, label_review_map, reviews, tombstone_release_year_map, tombstones
Thought:I can query the "genre_review_map" table to get the genres.
Action: sql_db_schema
Action Input: "genre_review_map"
Observation: 
CREATE TABLE genre_review_map (
	review_url VARCHAR NOT NULL, 
	genre VARCHAR NOT NULL, 
	PRIMARY KEY (review_url, genre), 
	FOREIGN KEY(review_url) REFERENCES reviews (review_url)
)

/*
3 rows from genre_review_map table:
review_url	genre
/reviews/albums/9232-vesikansi/	Experimental
/reviews/albums/9232-vesikansi/	Rock
/reviews/albums/22823-elsewhere/	Rock
*/
Thought:The genres can be found in the "genre_review_map" table.
Action: sql_db_query
Action Input: "SELECT DISTINCT genre FROM genre_review_map LIMIT 10"
Observation: [('Experimental',), ('Rock',), ('Electronic',), ('Folk/Country',), ('Rap',), ('Metal',), ('Pop/R&B',), (

'Experimental, Rock, Electronic, Folk/Country, Rap, Metal, Pop/R&B, Jazz, Global'

In [16]:
question = "Which is the genre with most reviews?"
agent_executor.run(question)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: artist_review_map, artists, author_review_map, genre_review_map, label_review_map, reviews, tombstone_release_year_map, tombstones
Thought:I see that there are tables related to reviews, genres, and artists. I should query the schema of these tables to see what columns are available.
Action: sql_db_schema
Action Input: "genre_review_map, genres"
Observation: Error: table_names {'genres'} not found in database
Thought:It seems that the table "genres" does not exist in the database. I should try querying the schema of the other relevant tables.
Action: sql_db_schema
Action Input: "genre_review_map, reviews, artists"
Observation: 
CREATE TABLE artists (
	artist_id VARCHAR NOT NULL, 
	name VARCHAR NOT NULL, 
	artist_url VARCHAR, 
	PRIMARY KEY (artist_id)
)

/*
3 rows from artists table:
artist_id	name	artist_url
4847-avarus	Avarus	/artists/4847-avarus/
33420-pinegrove	Pinegrove	/artists/33420-p

'Rock'

# Conclusions

We experimented the SQL Agent with a sqlite3 database on Kaggle.  
For this I am using at this point the ChatOpenAI model.   
The SQL Agent seems to be working well and provides accurate results.  
As a next step, I would like to check if it is possible to use Kaggle Models as a replacement for OpenAI LLLM to power the SQL Agent.  
To get more insights into this database, you can also consult this Notebook: https://www.kaggle.com/code/gpreda/explore-a-sqlite3-database 
